In [71]:
import pandas as pd
import glob
import numpy as np

In [72]:
file_pattern = "/share/costeffectiveness/artifacts/vivarium_nih_us_cvd/raw_data/heart_failure_paf_pop_size_profiling/*.hdf"

def get_age_and_sex(measure_str):
    age = measure_str.split('AGE_GROUP_')[1].split('SEX')[0][:-1]
    sex = measure_str.split('AGE_GROUP_')[1].split('SEX')[1][1:]
    
    return age + ',' + sex

In [101]:
plotting_data = pd.DataFrame()

for file_name in glob.glob(file_pattern):
    pop_size = file_name.split('.hdf')[0].split('output_')[1]
    df = pd.read_hdf(file_name)
    df = df[[col for col in df.columns if 'blood_pressure' in col]].T # either blood_pressure or high_body_mass_index
    df.columns = [f"draw_{i}" for i in range(25)]
    df = df.reset_index()
    df['demographics'] = df['index'].apply(get_age_and_sex)
    df = df.drop('index', axis=1)
    df[['age', 'sex']] = df['demographics'].str.split(',', expand=True)
    df = df.drop('demographics', axis=1)
    df['pop_size'] = pop_size
    plotting_data = pd.concat([plotting_data, df])

In [110]:
age = '95_plus'
sex = 'Female'

subset = plotting_data.query("age==@age & sex==@sex")

In [113]:
print(np.std(subset.query("pop_size=='1_000_000'")[[f"draw_{i}" for i in range(25)]].values))
print(np.std(subset.query("pop_size=='100_000'")[[f"draw_{i}" for i in range(25)]].values))
print(np.std(subset.query("pop_size=='10_000'")[[f"draw_{i}" for i in range(25)]].values))
print(np.std(subset.query("pop_size=='1000'")[[f"draw_{i}" for i in range(25)]].values))

0.07869190673061091
0.07952400993569868
0.07866689618332497
0.14820015097189046
